# Extract AOD for each OpenAQ measurement

## Import library

In [6]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import requests
from pprint import pp
from tqdm.notebook import tqdm
import requests
import time
from zkyhaxpy import io_tools, pd_tools, gis_tools
from datetime import datetime, timedelta
import os

## Prepare GeoDataframe of OpenAQ data

In [11]:
if not os.path.exists('../data/gdf_openaq.gpkg'):
    df_openaq = pd.read_csv('../data/openaqi_thailand_pm25_data.csv', sep=',' )
    df_openaq['measurement_id'] = df_openaq.index

    df_openaq['geometry'] = 'POINT (' + df_openaq['long'].astype(str) + ' ' + df_openaq['lat'].astype(str) + ')'
    gdf_openaq = gis_tools.df_to_gdf(df_openaq, 'geometry')

    gdf_openaq['datetime'] = gdf_openaq['date_utc'].apply(lambda date_utc: pd.Timestamp(date_utc).tz_convert("UTC"))
    gdf_openaq['year'] = gdf_openaq['date_utc'].str.slice(0, 4).astype(int)
    gdf_openaq['month'] = gdf_openaq['date_utc'].str.slice(5, 7).astype(int)
    del(gdf_openaq['date_utc'])
    gdf_openaq = gdf_openaq.set_index('measurement_id')
    gdf_openaq.to_file('../data/gdf_openaq.gpkg')

In [12]:
gdf_openaq

,locationId,location,parameter,value,unit,country,city,isMobile,isAnalysis,entity,sensorType,lat,long,geometry,datetime,year,month
measurement_id,,,,,,,,,,,,,,,,,
0,8893,Vientiane,pm25,10.000000,µg/m³,TH,NaN,False,NaN,Governmental Organization,reference grade,17.896122,102.640000,POINT (102.64000 17.89612),2019-06-30 00:00:00+00:00,2019,6
1,8893,Vientiane,pm25,18.000000,µg/m³,TH,NaN,False,NaN,Governmental Organization,reference grade,17.896122,102.640000,POINT (102.64000 17.89612),2019-06-29 23:00:00+00:00,2019,6
2,8893,Vientiane,pm25,17.000000,µg/m³,TH,NaN,False,NaN,Governmental Organization,reference grade,17.896122,102.640000,POINT (102.64000 17.89612),2019-06-29 22:00:00+00:00,2019,6
3,8893,Vientiane,pm25,12.000000,µg/m³,TH,NaN,False,NaN,Governmental Organization,reference grade,17.896122,102.640000,POINT (102.64000 17.89612),2019-06-29 21:00:00+00:00,2019,6
4,8893,Vientiane,pm25,15.000000,µg/m³,TH,NaN,False,NaN,Governmental Organization,reference grade,17.896122,102.640000,POINT (102.64000 17.89612),2019-06-29 20:00:00+00:00,2019,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218204,2694254,Soi Kheha Pattana 1,pm25,39.079167,µg/m³,TH,NaN,False,NaN,Private Organization,low-cost sensor,13.709941,100.576022,POINT (100.57602 13.70994),2024-04-02 04:00:00+00:00,2024,4
218205,2694254,Soi Kheha Pattana 1,pm25,39.387797,µg/m³,TH,NaN,False,NaN,Private Organization,low-cost sensor,13.709941,100.576022,POINT (100.57602 13.70994),2024-04-02 03:00:00+00:00,2024,4
218206,2694254,Soi Kheha Pattana 1,pm25,37.445286,µg/m³,TH,NaN,False,NaN,Private Organization,low-cost sensor,13.709941,100.576022,POINT (100.57602 13.70994),2024-04-02 02:00:00+00:00,2024,4


In [24]:
dir_extracted_aod_root = r'../../../data/aod/extracted/aod_055'
df_files_aod = io_tools.get_list_files(dir_extracted_aod_root, '.tif$', return_df=True)

df_files_aod['date'] = df_files_aod['file_nm'].str.slice(8, 18)
df_files_aod['time'] = df_files_aod['file_nm'].str.slice(19, 21) + ':' + df_files_aod['file_nm'].str.slice(21, 23)
df_files_aod['datetime'] = df_files_aod['date'] + 'T' + df_files_aod['time'] + ':00+00:00'
df_files_aod['datetime'] = df_files_aod['datetime'].apply(lambda datetime: pd.Timestamp(datetime).tz_convert("UTC"))
df_files_aod['year'] = df_files_aod['datetime'].dt.year
df_files_aod

Total of 73617 files have been listed.


,file_path,file_nm,folder_nm,file_ext,date,time,datetime,year
0,../../../data/aod/extracted/aod_055\h26v06\200...,aod_055-2000-02-24_0520.tif,2000,tif,2000-02-24,05:20,2000-02-24 05:20:00+00:00,2000
1,../../../data/aod/extracted/aod_055\h26v06\200...,aod_055-2000-02-25_0425.tif,2000,tif,2000-02-25,04:25,2000-02-25 04:25:00+00:00,2000
2,../../../data/aod/extracted/aod_055\h26v06\200...,aod_055-2000-02-26_0325.tif,2000,tif,2000-02-26,03:25,2000-02-26 03:25:00+00:00,2000
3,../../../data/aod/extracted/aod_055\h26v06\200...,aod_055-2000-02-26_0510.tif,2000,tif,2000-02-26,05:10,2000-02-26 05:10:00+00:00,2000
4,../../../data/aod/extracted/aod_055\h26v06\200...,aod_055-2000-02-27_0410.tif,2000,tif,2000-02-27,04:10,2000-02-27 04:10:00+00:00,2000
...,...,...,...,...,...,...,...,...
73612,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0200.tif,2024,tif,2024-04-09,02:00,2024-04-09 02:00:00+00:00,2024
73613,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0340.tif,2024,tif,2024-04-09,03:40,2024-04-09 03:40:00+00:00,2024
73614,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0550.tif,2024,tif,2024-04-09,05:50,2024-04-09 05:50:00+00:00,2024
73615,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0725.tif,2024,tif,2024-04-09,07:25,2024-04-09 07:25:00+00:00,2024


In [28]:
df_files_aod = df_files_aod[df_files_aod['year'] >= 2024]
df_files_aod

,file_path,file_nm,folder_nm,file_ext,date,time,datetime,year
9201,../../../data/aod/extracted/aod_055\h26v06\202...,aod_055-2024-01-01_0345.tif,2024,tif,2024-01-01,03:45,2024-01-01 03:45:00+00:00,2024
9202,../../../data/aod/extracted/aod_055\h26v06\202...,aod_055-2024-01-01_0640.tif,2024,tif,2024-01-01,06:40,2024-01-01 06:40:00+00:00,2024
9203,../../../data/aod/extracted/aod_055\h26v06\202...,aod_055-2024-01-01_0820.tif,2024,tif,2024-01-01,08:20,2024-01-01 08:20:00+00:00,2024
9204,../../../data/aod/extracted/aod_055\h26v06\202...,aod_055-2024-01-02_0245.tif,2024,tif,2024-01-02,02:45,2024-01-02 02:45:00+00:00,2024
9205,../../../data/aod/extracted/aod_055\h26v06\202...,aod_055-2024-01-02_0425.tif,2024,tif,2024-01-02,04:25,2024-01-02 04:25:00+00:00,2024
...,...,...,...,...,...,...,...,...
73612,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0200.tif,2024,tif,2024-04-09,02:00,2024-04-09 02:00:00+00:00,2024
73613,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0340.tif,2024,tif,2024-04-09,03:40,2024-04-09 03:40:00+00:00,2024
73614,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0550.tif,2024,tif,2024-04-09,05:50,2024-04-09 05:50:00+00:00,2024
73615,../../../data/aod/extracted/aod_055\h28v08\202...,aod_055-2024-04-09_0725.tif,2024,tif,2024-04-09,07:25,2024-04-09 07:25:00+00:00,2024


In [29]:
MAX_DIFF_HOURS = 1
for s_idx, s_row in tqdm(df_files_aod.iterrows(), total=len(df_files_aod)):
    min_datetime = s_row.datetime - timedelta(hours=MAX_DIFF_HOURS)
    max_datetime = s_row.datetime + timedelta(hours=MAX_DIFF_HOURS)
    gdf_openaq_tmp = gdf_openaq.copy()
    gdf_openaq_tmp = gdf_openaq_tmp[gdf_openaq_tmp.datetime.between(min_datetime, max_datetime)]
    if len(gdf_openaq_tmp) > 0:
        df_extracted_aod055_curr = gis_tools.extract_pixval_single_file(
            in_s_polygon=gdf_openaq_tmp['geometry'] ,
            in_raster_path=s_row.file_path,
            in_list_out_col_nm=['aod_055'],
            in_list_target_raster_band_id=[1],
            nodata_val=-28762
            )

        break
    
        
    

  0%|          | 0/2398 [00:00<?, ?it/s]

c:tmp\4ee9ec5e6cb34a9f8e1f4b47e5fd45b6\rowcol_map_aod_055-2024-04-01_0305.tif has been created


AttributeError: 'GeoSeries' object has no attribute 'iteritems'

In [20]:
df_files_aod

,file_path,file_nm,folder_nm,file_ext,date,time,datetime
0,../../../data/aod/extracted\aod_055\h26v06\200...,aod_055-2000-02-24_0520.tif,2000,tif,2000-02-24,05:20,2000-02-24T05:20:00+00:00
1,../../../data/aod/extracted\aod_055\h26v06\200...,aod_055-2000-02-25_0425.tif,2000,tif,2000-02-25,04:25,2000-02-25T04:25:00+00:00
2,../../../data/aod/extracted\aod_055\h26v06\200...,aod_055-2000-02-26_0325.tif,2000,tif,2000-02-26,03:25,2000-02-26T03:25:00+00:00
3,../../../data/aod/extracted\aod_055\h26v06\200...,aod_055-2000-02-26_0510.tif,2000,tif,2000-02-26,05:10,2000-02-26T05:10:00+00:00
4,../../../data/aod/extracted\aod_055\h26v06\200...,aod_055-2000-02-27_0410.tif,2000,tif,2000-02-27,04:10,2000-02-27T04:10:00+00:00
...,...,...,...,...,...,...,...
73300,../../../data/aod/extracted\aod_055\h28v08\202...,aod_055-2024-04-09_0725.tif,2024,tif,2024-04-09,07:25,2024-04-09T07:25:00+00:00
73301,../../../data/aod/extracted\aod_055\h28v08\202...,aod_055-2024-04-09_0730.tif,2024,tif,2024-04-09,07:30,2024-04-09T07:30:00+00:00
73302,../../../data/aod/extracted\aod_055_test\h26v0...,aod_055_test-2024-01-01_0345.tif,2024,tif,test-2024-,1-:01,test-2024-T1-:01:00+00:00
73303,../../../data/aod/extracted\aod_055_test\h26v0...,aod_055_test-2024-01-01_0640.tif,2024,tif,test-2024-,1-:01,test-2024-T1-:01:00+00:00
